##OpenSearch OSDD Analysis

In [2]:
%matplotlib inline
import pandas as pd
import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker

pd.set_option("display.max_rows",250)

In [3]:
# grab the clean text from the rds
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))

So what do we see in the 4906 responses?

In [7]:
sql = """select kvps.value, count(kvps.value)
from osdds,
    json_each_text(namespaces) as kvps
group by kvps.value
order by count desc, kvps.value
;"""

In [ ]:
# df = pd.read_sql(sql, engine, index_col='id', parse_dates=['metadata_age'])

In [8]:
df = pd.read_sql(sql, engine)

In [9]:
df
# number of references to a namespace

,value,count
0,http://a9.com/-/spec/opensearch/1.1/,4614
1,http://www.w3.org/XML/1998/namespace,4608
2,http://www.mozilla.org/2006/browser/search/,1053
3,http://a9.com/-/opensearch/extensions/geo/1.0/,69
4,http://a9.com/-/opensearch/extensions/time/1.0/,32
5,http://www.w3.org/2001/XMLSchema,19
6,http://www.w3.org/2001/XMLSchema-instance,19
7,http:/www.mozilla.org/2006/browser/search/,10
8,http://modwebsrv.modaps.eosdis.nasa.gov/opense...,9
9,http://a9.com/-/opensearch/extensions/referrer/,8


In [ ]:
# how many contain 